In [1]:
from PIL import Image
import glob
import numpy as np
import os
from os.path import basename

yTrain = []
for filename in glob.glob('Dataset/*.png'): #assuming png
    im = Image.open(filename)
    label = (filename.split("_")[1])
    #print(label)
    yTrain.append(label)
    im.close()

#print(yTrain)

In [2]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

flat_Image_List = [] # new list containing flatten image arrays
#for img in yTrain: # iterate over the images contained in the directory
for filename in glob.glob('Dataset/*.png'):
    im = Image.open(filename)
    img = np.reshape(im, (12288))
    flat_Image_List.append(img) # Append to the list 
    #plt.imshow(img)
    im.close()
    #print(flat_Image_List)
#print(flat_Image_List)

In [3]:
imageSize = 12288

In [4]:
from numpy import array

xTrain = array(flat_Image_List)

#print(flat_Image_List)
print(xTrain)

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


In [5]:
# Converts RGB to a range between 0-1
xTrain = xTrain/255

In [ ]:
xTrain = np.reshape(xTrain,(len(xTrain), 64, 64, 3))

In [ ]:
xTrain = xTrain.astype('float32')

In [6]:
noLabels = len(np.unique(yTrain))
print(noLabels)
yTrain = np.array(yTrain)
print(yTrain.shape)

15
(73055,)


In [7]:
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

data = ['0', '1', '2','3', '4','5', '6','7', '8', '9', 'A', 'B', 'C', 'D', 'E']

values = array(data)
print(values)

['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' 'A' 'B' 'C' 'D' 'E']


In [8]:
# integer encode
label_encoder = LabelEncoder()
labelData = label_encoder.fit_transform(yTrain)
print(labelData)

[ 0  0  0 ... 14 14 14]


In [9]:
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
labelData = labelData.reshape(len(labelData), 1)
labelData = onehot_encoder.fit_transform(labelData)
print(labelData)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [11]:
# invert example
inverted = label_encoder.inverse_transform([argmax(labelData[10, :])])
print(inverted)

['0']


C:\Users\niall\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [12]:
yTrain.shape
print(yTrain)

['0' '0' '0' ... 'E' 'E' 'E']


In [14]:
input_shape = (64, 64, 3)

In [16]:
# For encoding categorical variables and pre processing.
import sklearn.preprocessing as pre
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

encoder = pre.LabelBinarizer()
encoder.fit(yTrain)
outputs = encoder.transform(yTrain)


inputs = xTrain
inputs.shape
print(encoder)

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)


In [17]:
# ------- MODEL -------
# Import keras.
import keras as kr
import tensorflow as tf

# Importing the required Keras modules containing model and layers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
# Adapted from: https://towardsdatascience.com/image-classification-in-10-minutes-with-mnist-dataset-54c35b77a38d?fbclid=IwAR01njT_lhc2ZZOySJnWhmq8z9iWUcKjefacuRj_bI1rJbmR0NCW1cr-ao4
# Start a neural network, building it by layers.
model = kr.models.Sequential()
# Add a hidden layer with 784 neurons.
model.add(Conv2D(64, kernel_size=(3,3), input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten()) # Flattening the 2D arrays for fully connected layers
model.add(Dense(256, activation=tf.nn.relu))
model.add(Dropout(0.4))
model.add(Dense(50,activation=tf.nn.relu))

# Add a 15 neuron output layer.
model.add(Dense(15, activation='softmax'))

model.summary()

C:\Users\niall\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 64)        1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 61504)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               15745280  
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                12850     
_________________________________________________________________
dense_3 (Dense)              (None, 15)                765       
Total para

In [18]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Number of Epoch is the amount of times the training set is put through the model
# The batch size is the amount of images the models processes at one time
model.fit(xTrain,labelData, epochs=10, batch_size=300)

Epoch 1/10
73055/73055 [==============================] - 979s 13ms/step - loss: 2.2771 - acc: 0.3021
Epoch 2/10
73055/73055 [==============================] - 685s 9ms/step - loss: 1.4432 - acc: 0.4957
Epoch 3/10
73055/73055 [==============================] - 660s 9ms/step - loss: 1.1793 - acc: 0.5885
Epoch 4/10
73055/73055 [==============================] - 684s 9ms/step - loss: 0.7160 - acc: 0.7662
Epoch 5/10
13800/73055 [====>.........................] - ETA: 9:04 - loss: 0.5128 - acc: 0.8387

KeyboardInterrupt: 

In [ ]:
scores = model.evaluate(xTest, yTest, verbose = 0)

print("Error percentage: %.2f%%" %(100 - score[1] * 100))